# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [6]:
#Your code here
import pandas as pd
df=pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
df.columns

Index(['timestamp', 'id', 'group', 'action'], dtype='object')

In [8]:
df1 = df['action']
unique_values = df1.unique()
unique_values

array(['view', 'click'], dtype=object)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [13]:
from scipy.stats import ttest_ind
# Filter the data to separate the experimental and control groups
exp_data = df[df['group'] == 'experiment']
ctl_data = df[df['group'] == 'control']

# Calculate the click-through rate of each group
exp_views = exp_data[exp_data['action'] == 'view']['action'].count()
exp_clicks = exp_data[exp_data['action'] == 'click']['action'].count()
exp_ctr = exp_clicks / exp_views

ctl_views = ctl_data[ctl_data['action'] == 'view']['action'].count()
ctl_clicks = ctl_data[ctl_data['action'] == 'click']['action'].count()
ctl_ctr = ctl_clicks / ctl_views

# Perform the two-sample t-test
t_statistic, p_value = ttest_ind(exp_data[exp_data['action'] == 'click']['id'], ctl_data[ctl_data['action'] == 'click']['id'], equal_var=False)

# Print the results
print(f"Experimental group click-through rate: {exp_ctr:.3f}")
print(f"Control group click-through rate: {ctl_ctr:.3f}")
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

if exp_ctr > ctl_ctr:
    print("The experimental homepage was more effective than the control homepage.")
else:
    print("The control homepage was more effective than the experimental homepage.")


Experimental group click-through rate: 0.310
Control group click-through rate: 0.280
t-statistic: 0.583
p-value: 0.560
The experimental homepage was more effective than the control homepage.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [16]:
#Your code here
#create a binary variable for each individual indicating whether or not they clicked on the homepage
df["clicked"] = df["action"].apply(lambda x: 1 if x == "click" else 0)
# multiplying the total number of individuals in the experiment group by the click-through
#rate of the control group.
control_ctr = df.query("group == 'control'")["clicked"].mean()
experiment_size = df.query("group == 'experiment'").shape[0]
#the expected number of clicks for the experiment group if it had the same click-through rate as 
#that of the control group
expected_clicks = control_ctr * experiment_size
expected_clicks

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [18]:
#Your code here
import numpy as np

experiment_clicks = df.query("group == 'experiment'")["clicked"].sum()
variance = experiment_size * control_ctr * (1 - control_ctr)
std_dev = np.sqrt(variance)
z_score = (experiment_clicks - expected_clicks) / std_dev
z_score

2.71607302278784

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [19]:
#Your code here
from scipy.stats import norm

p_value = norm.sf(abs(z_score)) * 2  # two-tailed test
p_value

0.006606134551853142

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here** Yes, the result obtained from the binomial test matches the result obtained from the two-sample t-test. Both tests indicate that the experimental homepage is more effective than the control homepage. In both cases, the p-value is less than the significance level of 0.05, which suggests that the difference in click-through rates between the two groups is statistically significant. The difference in proportions calculated using the binomial test also confirms that the experimental homepage is more effective than the control homepage, as the difference is positive and the confidence interval does not include 0.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.